In [ ]:
# pip install virtualenv

# python -m venv <envname> 

# cd .\myenv1\  

# .\Scripts\activate

create one file name main.py 

#pip install fastapi uvicorn


In [ ]:
main.py

from fastapi import FastAPI

app = FastAPI() 

@app.get("/")
def greet():
    return "Good Morning"


//// for run below commands need to execute 
# uvicorn main:app --reload

In [ ]:
open your browser : http://127.0.0.1:8000/

In [ ]:
=>  Fetch Api Example - Fetch All records : 

Retrive records from specific record 

here we are going make one dummy model class in model.py 
file at same location 

model.py
------------------------------------------------------
class Product:
    id : int 
    name : str 
    description : str 
    price : int 
    qty : int 
    
    def __init__(self,id:int,name:str,description,price,qty):
        self.id = id
        self.name = name
        self.description =description
        self.price = price
        self.qty = qty

In [ ]:
now, time to call this model in main.py file 

main.py

from fastapi import FastAPI
from models import * 
app = FastAPI() 

products = [
    Product(1,"Mobile","Iphone 16",68000,20),
    Product(2,"Laptop","Dell",57000,40),
    Product(3,"AC","Iphone 18",68000,30)
]

@app.get("/products")
def get_all_products():
    return products

In [ ]:
now run : 

http://127.0.0.1:8000/products


expected output like ::

[
  {
    "id": 1,
    "name": "Mobile",
    "description": "Iphone 16",
    "price": 68000,
    "qty": 20
  },
  .
  .
]

In [ ]:
Pydantic is a Python library used for data validation and 
data parsing using type hints.

in simple language we can say that 

Pydantic checks if your data is correct or not.

e.g.

Suppose user sends this data to your API

{
  "id": "abc",
  "price": "hello"
}

But your app expects:

id → int

price → int

-> Without validation  :  X 
Your app may crash.

-> With Pydantic
It automatically checks types and throws proper error

In [ ]:
What is BaseModel?

BaseModel is the main class from Pydantic.

importing the core validation engine.
e.g.
    from pydantic import BaseModel

In [ ]:
when we use pydantic and BaseModel we don't need to 
create our separate constructor or __init__() method 

e.g.

from pydantic import BaseModel

class Product(BaseModel):
    id : int 
    name : str 
    description : str 
    price : int 
    qty : int 

    # not required below method 
    # def __init__(self,id:int,name:str,description:str,price:int,qty:int):
    #     self.id = id
    #     self.name = name
    #     self.description =description
    #     self.price = price
    #     self.qty = qty

In [ ]:
FastApi Provide us 

http://127.0.0.1:8000/docs

In [ ]:
=> Fetch product by specific id 

main.py 

@app.get("/products/{id}")
def get_specific_product(id : int):
    return products[id-1]


http://127.0.0.1:8000/products/1

expected output : 

{"id":1,"name":"Mobile","description":"Iphone 16","price":68000,"qty":20}


In [ ]:
=> For more validation we can follow below code : 

@app.get("/products/{id}")
def get_specific_product(id : int):
    for product in products:
        if product.id == id:
            return product
    
    return "no records found !"

In [ ]:
=> Add new record using api 

-> Using of post method 

@app.post("/products")
def add_product(product:Product):
    products.append(product)
    return product


In [ ]:
=> for update and delete record 


@app.put("/products")
def update_product(id : int,product:Product):
    for i in range(len(products)):
        if products[i].id == id:
            products[i] = product
            return "detailed updated !!"
    return "no records found !"


@app.delete("/products")
def delete_product(id : int):
    for i in range(len(products)):
        if products[i].id == id:
            del products[i]
            return "Product deleted !!!"
    return "no product found !!"